#9.1. Exploring Threadblock-Level Groups

##1a. Creating Groups

First, you should take the task1.cu code, and complete the sections indicated by FIXME to provide a proper thread-block group, and assign that group to the group being used for printout purposes. You should only need to modify the 2 lines containing FIXME for this first step.

You can compile your code as follows:

nvcc -arch=sm_70 -o task1 task1.cu -std=c++11

nvcc is the CUDA compiler invocation command. The syntax is generally similar to gcc/g++. Note that because we're using C++11 (which is required for cooperative groups) we need a sufficiently modern compiler (gcc >= 5 should be sufficient).

Correct output should look like this:

```
group partial sum: 256
```
If you need help, refer to the task1_solution1.cu file. (which contains the solution for tasks 1a, 1b, and 1c)

##1b. Partitioning Groups

Next uncomment the next line that starts with the auto keyword, and complete that line to use the previously created thread block group and subdivide it into a set of 32-thread partitions, using the dynamic (runtime) partitioning method.

Compile and run the code as above. correct output should look like:

```
group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32
```
##1c. Third Group Creation/Decomposition

Now perform the 3rd group creation/decomposition.

Compile and run the code as above. Correct output should look like:

```
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
```

##task_1.cu

```
#include <cooperative_groups.h>
#include <stdio.h>
using namespace cooperative_groups;
const int nTPB = 256;
__device__ int reduce(thread_group g, int *x, int val) {
  int lane = g.thread_rank();
  for (int i = g.size()/2; i > 0; i /= 2) {
    x[lane] = val;       g.sync();
    if (lane < i) val += x[lane + i];  g.sync();
  }
  if (g.thread_rank() == 0) printf("group partial sum: %d\n", val);
  return val;
}

__global__ void my_reduce_kernel(int *data){

  __shared__ int sdata[nTPB];
  // task 1a: create a proper thread block group below
  auto g1 = FIXME
  size_t gindex = g1.group_index().x * nTPB + g1.thread_index().x;
  // task 1b: uncomment and create a proper 32-thread tile below, using group g1 created above
  // auto g2 = FIXME
  // task 1c: uncomment and create a proper 16-thread tile below, using group g2 created above
  // auto g3 = FIXME
  // for each task, adjust the group to point to the last group created above
  auto g = FIXME
  // Make sure we send in the appropriate patch of shared memory
  int sdata_offset = (g1.thread_index().x / g.size()) * g.size();
  reduce(g, sdata + sdata_offset, data[gindex]);
}

int main(){

  int *data;
  cudaMallocManaged(&data, nTPB*sizeof(data[0]));
  for (int i = 0; i < nTPB; i++) data[i] = 1;
  my_reduce_kernel<<<1,nTPB>>>(data);
  cudaError_t err = cudaDeviceSynchronize();
  if (err != cudaSuccess) printf("cuda error: %s\n", cudaGetErrorString(err));
}

```

In [5]:
%%writefile task_1c.cu
# include <cooperative_groups.h>
# include <stdio.h>
using namespace cooperative_groups;
const int nTPB = 256;
__device__ int reduce(thread_group g, int *x, int val) {
  int lane = g.thread_rank();
  for (int i = g.size()/2; i > 0; i /= 2) {
    x[lane] = val;       g.sync();
    if (lane < i) val += x[lane + i];  g.sync();
  }
  if (g.thread_rank() == 0) printf("group partial sum: %d\n", val);
  return val;
}

__global__ void my_reduce_kernel(int *data){

  __shared__ int sdata[nTPB];
  // task 1a: create a proper thread block group below
  auto g1 = this_thread_block();
  size_t gindex = g1.group_index().x * nTPB + g1.thread_index().x;
  // task 1b: uncomment and create a proper 32-thread tile below, using group g1 created above
  auto g2 = tiled_partition(g1,32);
  // task 1c: uncomment and create a proper 16-thread tile below, using group g2 created above
  auto g3 = tiled_partition(g2,16);
  // for each task, adjust the group to point to the last group created above
  auto g = g3;
  // Make sure we send in the appropriate patch of shared memory
  int sdata_offset = (g1.thread_index().x / g.size()) * g.size();
  reduce(g, sdata + sdata_offset, data[gindex]);
}

int main(){

  int *data;
  cudaMallocManaged(&data, nTPB*sizeof(data[0]));
  for (int i = 0; i < nTPB; i++) data[i] = 1;
  my_reduce_kernel<<<1,nTPB>>>(data);
  cudaError_t err = cudaDeviceSynchronize();
  if (err != cudaSuccess) printf("cuda error: %s\n", cudaGetErrorString(err));
}


Writing task_1c.cu


In [5]:
!nvcc -arch=sm_70 -o task_1 task_1.cu -std=c++11

In [6]:
!./task_1

group partial sum: 256


In [3]:
!nvcc -arch=sm_70 -o task_1b task_1b.cu -std=c++11

In [4]:
!./task_1b


group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32
group partial sum: 32


In [6]:
!nvcc -arch=sm_70 -o task_1c task_1c.cu

In [7]:
!./task_1c

group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16
group partial sum: 16


#9.2 Exploring Grid-Wide Sync

One of the motivations suggested for a grid-wide sync is to combine algorithm phases which need to be completed in sequence, and would normally be realized with separate CUDA kernel calls. In this case, the kernel launch boundary provides an implicit/effective grid-wide sync. However cooperative groups provides the possibility of a grid wide sync directly in kernel code, rather than at a kernel launch boundary.

One such algorithm would be stream compaction. Stream compaction is used in many places, and fundamentally seeks to reduce the length of a data stream using a particular removal heuristic or predicate test. For example, if we had the following data stream:

```
3 4 3 7 0 5 0 8 0 0 0 4
```

we could do stream compaction by removing the zeroes, ending up with:

```
3 4 3 7 5 8 4
```
Like many reduction type algorithms (the output here is potentially much smaller than the input), we can easily imagine how to do this in a serial fashion, but a fast parallel stream compaction requires some additional thought. A common approach is to use a prefix sum. A prefix sum is a data set, where each data item in the set represents the sum of the previous input elements from the beginning of the input to that point. We can use a prefix sum to help parallelize our stream compaction. We start by creating an array of ones and zeroes, where there is a one corresponding to the element we want to keep, and zero for the element we want to discard:

```
3 4 3 7 0 5 0 8 0 0 0 4 (input data)
1 1 1 1 0 1 0 1 0 0 0 1 (filtering of input)
```

We then do an exclusive prefix sum on that filtered array (exclusive means only the elements "to the left" are included in the sum. The element at that position is excluded).

```
3 4 3 7 0 5 0 8 0 0 0 4 (input data)
1 1 1 1 0 1 0 1 0 0 0 1 (filtering of input)
0 1 2 3 4 4 5 5 6 6 6 6 (exclusive prefix sum of filtered data)
```

This prefix sum now contains the index into the output array that the input position should be copied to. We only copy a position from input to output if the corresponding filter element is not zero. This demonstrates how to use a prefix sum to assist with a stream compaction, but doesn't identify how to do the prefix sum in parallel, efficiently. A full treatment here is beyond the scope of this document, but you can refer here for a good treatise: https://people.eecs.berkeley.edu/~driscoll/cs267/papers/gpugems3_ch39.html Some key takeaways are that a prefix sum has a sweeping operation, not unlike the sweeping operation that is successively performed in a parallel reduction, but there are key differences. Two of these key differences are that the sweep is from "left" to "right" in the prefix sum whereas it is usually from right to left in a typical parallel reduction, and also that the break points (i.e. the division of threads participating at each sweep phase) is different.

When parallelizing a prefix sum, we often require multiple phases, for example a thread-block level scan (prefix-sum) operation, followed by another operation to "fix up" the threadblock level results based on the data from other ("previous") thread blocks. These phases may require a grid-wide sync, and typical scan from a library such as thrust will use multiple kernel calls. Let's see if we can do it in a single kernel call. You won't have to write any scan code, other than inserting appropriate cooperative group sync points. We need sync points at the threadblock leve (based on the threadblock level group created for you) and also at the grid level.

Start with the task2.cu code, and perform 2 things:

Modify the FIXME statements in the kernel to insert appropriate sync operations as requested, based on the two group types created at the top of the kernel. Only one grid-wide sync point is needed, the others are all thread-block-level sync points.
In the host code, modify the FIXME statements to do a proper cooperative launch. The launch function is already provided, you just need to fill in the remaining 4 arguments. Refer to the task2_solution.cu file for help, or refer to the cuda runtime API documentation for the launch function: https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__EXECUTION.html#group__CUDART__EXECUTION_1g504b94170f83285c71031be6d5d15f73
Once you have made the above modification, compile your code as follows: